In [0]:
-- ============================================
-- Databricks SQL Analysis for Railway Data
-- Compatible with Databricks SQL & Delta Lake
-- ============================================


-- ============================================
-- SECTION 1: OVERVIEW METRICS
-- ============================================

-- Total bookings and revenue
SELECT 
    COUNT(*) AS total_bookings,
    SUM(Price) AS total_revenue,
    AVG(Price) AS avg_ticket_price,
    MIN(Price) AS min_price,
    MAX(Price) AS max_price
FROM workspace.default.railway;

-- ============================================
-- SECTION 2: JOURNEY PERFORMANCE
-- ============================================

-- Journey status breakdown
SELECT 
    `Journey Status`,
    COUNT(*) AS count,
    ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER(), 2) AS percentage
FROM workspace.default.railway
GROUP BY `Journey Status`
ORDER BY count DESC;

-- Delay reasons analysis
SELECT 
    `Reason for Delay`,
    COUNT(*) AS incident_count,
    ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM workspace.default.railway WHERE `Reason for Delay` IS NOT NULL), 2) AS pct_of_delays
FROM workspace.default.railway
WHERE `Reason for Delay` IS NOT NULL
GROUP BY `Reason for Delay`
ORDER BY incident_count DESC;

-- ============================================
-- SECTION 3: REVENUE ANALYSIS
-- ============================================

-- Revenue by ticket type
SELECT 
    `Ticket Type`,
    SUM(Price) AS total_revenue,
    AVG(Price) AS avg_price,
    COUNT(*) AS ticket_count,
    ROUND(SUM(Price) * 100.0 / (SELECT SUM(Price) FROM workspace.default.railway), 2) AS revenue_share_pct
FROM workspace.default.railway
GROUP BY `Ticket Type`
ORDER BY total_revenue DESC;

-- Revenue by ticket class
SELECT 
    `Ticket Class`,
    SUM(Price) AS total_revenue,
    AVG(Price) AS avg_price,
    COUNT(*) AS ticket_count
FROM workspace.default.railway
GROUP BY `Ticket Class`
ORDER BY total_revenue DESC;

-- Revenue by ticket type and class
SELECT 
    `Ticket Type`,
    `Ticket Class`,
    SUM(Price) AS total_revenue,
    AVG(Price) AS avg_price,
    COUNT(*) AS ticket_count
FROM workspace.default.railway
GROUP BY `Ticket Type`, `Ticket Class`
ORDER BY `Ticket Type`, `Ticket Class`;

-- ============================================
-- SECTION 4: STATION PERFORMANCE
-- ============================================

-- Top departure stations
SELECT 
    `Departure Station`,
    COUNT(*) AS departure_count,
    SUM(Price) AS revenue_generated
FROM workspace.default.railway
GROUP BY `Departure Station`
ORDER BY departure_count DESC
LIMIT 10;

-- Station delay rates (min 100 bookings)
SELECT 
    `Departure Station`,
    COUNT(*) AS total_bookings,
    SUM(CASE WHEN `Journey Status` = 'Delayed' THEN 1 ELSE 0 END) AS delayed_count,
    ROUND(SUM(CASE WHEN `Journey Status` = 'Delayed' THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) AS delay_rate_pct
FROM workspace.default.railway
GROUP BY `Departure Station`
HAVING COUNT(*) >= 100
ORDER BY delay_rate_pct DESC
LIMIT 10;

-- ============================================
-- SECTION 5: ROUTE ANALYSIS
-- ============================================

-- Top routes by bookings
SELECT 
    `Departure Station`,
    `Arrival Destination`,
    CONCAT(`Departure Station`, ' → ', `Arrival Destination`) AS route,
    COUNT(*) AS booking_count,
    AVG(Price) AS avg_price,
    SUM(Price) AS total_revenue
FROM workspace.default.railway
GROUP BY `Departure Station`, `Arrival Destination`
ORDER BY booking_count DESC
LIMIT 15;

-- Most expensive routes (min 50 bookings)
SELECT 
    CONCAT(`Departure Station`, ' → ', `Arrival Destination`) AS route,
    COUNT(*) AS booking_count,
    Round(AVG(Price),2) AS avg_price,
    MAX(Price) AS max_price
FROM workspace.default.railway
GROUP BY `Departure Station`, `Arrival Destination`
HAVING COUNT(*) >= 50
ORDER BY avg_price DESC
LIMIT 10;

-- ============================================
-- SECTION 6: CUSTOMER BEHAVIOR
-- ============================================

-- Purchase type distribution
SELECT 
    `Purchase Type`,
    COUNT(*) AS count,
    ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER(), 2) AS percentage
FROM workspace.default.railway
GROUP BY `Purchase Type`
ORDER BY count DESC;

-- Payment method preferences
SELECT 
    `Payment Method`,
    COUNT(*) AS count,
    ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER(), 2) AS percentage
FROM workspace.default.railway
GROUP BY `Payment Method`
ORDER BY count DESC;

-- Railcard usage
SELECT 
      -- if the value is null for any row ,he value 'None' will be shown instead
    COALESCE(Railcard, 'None') AS railcard_type,
    COUNT(*) AS count,
    ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER(), 2) AS percentage,
    Round(AVG(Price),2) AS avg_price
FROM workspace.default.railway
GROUP BY Railcard
ORDER BY count DESC;

-- ============================================
-- SECTION 7: REFUND ANALYSIS
-- ============================================

-- Refund requests by journey status
SELECT 
    `Journey Status`,
    COUNT(*) AS total_journeys,
    SUM(CASE WHEN `Refund Request` = 'Yes' THEN 1 ELSE 0 END) AS refund_requests,
    ROUND(SUM(CASE WHEN `Refund Request` = 'Yes' THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) AS refund_rate_pct
FROM workspace.default.railway
GROUP BY `Journey Status`
ORDER BY refund_rate_pct DESC;

-- Revenue at risk from cancellations
SELECT 
    `Journey Status`,
    COUNT(*) AS journey_count,
    SUM(Price) AS revenue_at_risk,
    ROUND(SUM(Price) * 100.0 / (SELECT SUM(Price) FROM workspace.default.railway), 2) AS pct_of_total_revenue
FROM workspace.default.railway
WHERE `Journey Status` IN ('Cancelled', 'Delayed')
GROUP BY `Journey Status`;

-- ============================================
-- SECTION 8: TIME-BASED ANALYSIS
-- ============================================

-- Bookings by hour of departure
SELECT 
    HOUR(`Departure Time`) AS departure_hour,
    COUNT(*) AS booking_count,
    ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER(), 2) AS percentage
FROM workspace.default.railway
WHERE `Departure Time` IS NOT NULL
GROUP BY HOUR(`Departure Time`)
ORDER BY departure_hour;

-- Delay rate by hour
SELECT 
    HOUR(`Departure Time`) AS departure_hour,
    COUNT(*) AS total_bookings,
    SUM(CASE WHEN `Journey Status` = 'Delayed' THEN 1 ELSE 0 END) AS delayed_count,
    ROUND(SUM(CASE WHEN `Journey Status` = 'Delayed' THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) AS delay_rate_pct
FROM workspace.default.railway
WHERE `Departure Time` IS NOT NULL
GROUP BY HOUR(`Departure Time`)
ORDER BY departure_hour;

-- ============================================
-- SECTION 9: DASHBOARD VIEWS
-- ============================================

-- Created view for executive dashboard
CREATE OR REPLACE TEMPORARY VIEW vw_executive_summary AS
SELECT 
    COUNT(*) AS total_bookings,
    SUM(Price) AS total_revenue,
    AVG(Price) AS avg_ticket_price,
    SUM(CASE WHEN `Journey Status` = 'On Time' THEN 1 ELSE 0 END) AS on_time_count,
    SUM(CASE WHEN `Journey Status` = 'Delayed' THEN 1 ELSE 0 END) AS delayed_count,
    SUM(CASE WHEN `Journey Status` = 'Cancelled' THEN 1 ELSE 0 END) AS cancelled_count,
    ROUND(SUM(CASE WHEN `Journey Status` = 'On Time' THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) AS on_time_pct,
    ROUND(SUM(CASE WHEN `Journey Status` = 'Delayed' THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) AS delay_pct,
    ROUND(SUM(CASE WHEN `Journey Status` = 'Cancelled' THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) AS cancellation_pct,
    SUM(CASE WHEN `Refund Request` = 'Yes' THEN 1 ELSE 0 END) AS refund_requests,
    ROUND(SUM(CASE WHEN `Refund Request` = 'Yes' THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) AS refund_rate_pct
FROM workspace.default.railway;

-- Created view for station performance
CREATE OR REPLACE TEMPORARY VIEW vw_station_performance AS
SELECT 
    `Departure Station`,
    COUNT(*) AS total_bookings,
    SUM(Price) AS total_revenue,
    AVG(Price) AS avg_price,
    SUM(CASE WHEN `Journey Status` = 'Delayed' THEN 1 ELSE 0 END) AS delayed_count,
    SUM(CASE WHEN `Journey Status` = 'Cancelled' THEN 1 ELSE 0 END) AS cancelled_count,
    ROUND(SUM(CASE WHEN `Journey Status` = 'Delayed' THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) AS delay_rate_pct,
    ROUND(SUM(CASE WHEN `Journey Status` = 'Cancelled' THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) AS cancellation_rate_pct
FROM workspace.default.railway
GROUP BY `Departure Station`;

-- Created view for route analysis
CREATE OR REPLACE TEMPORARY VIEW vw_route_analysis AS
SELECT 
    `Departure Station`,
    `Arrival Destination`,
    CONCAT(`Departure Station`, ' → ', `Arrival Destination`) AS route,
    COUNT(*) AS booking_count,
    SUM(Price) AS total_revenue,
    AVG(Price) AS avg_price,
    SUM(CASE WHEN `Journey Status` = 'Delayed' THEN 1 ELSE 0 END) AS delayed_count,
    ROUND(SUM(CASE WHEN `Journey Status` = 'Delayed' THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) AS delay_rate_pct
FROM workspace.default.railway
GROUP BY `Departure Station`, `Arrival Destination`;

-- ============================================
-- SECTION 10: EXECUTIVE SUMMARY QUERY
-- ============================================

SELECT * FROM vw_executive_summary;
select * from vw_route_analysis;
select * from vw_station_performance